In [1]:
import pandas as pd
import numpy as np
from numpy.random import multivariate_normal as Gaussian
import time
from scipy.stats import logistic, truncnorm, multivariate_normal
from termcolor import colored

In [2]:
def sigma(y):
    O = np.zeros(y.shape)
    I = (y>=0)
    J = (y<0)
    O[I] = 1/(1+np.exp(-y[I]))
    O[J] = np.exp(y[J])/(1+np.exp(y[J]))
    return O

In [3]:
sigma__ = lambda z: logistic.cdf(z)

In [4]:
data = np.genfromtxt("data/bank-note/train.csv", dtype = float, delimiter = ',')
data.shape

(872, 5)

In [5]:
data = np.hstack((np.ones((data.shape[0],1)), data))
data.shape

(872, 6)

In [6]:
data_train = data[:,:-1]
label_train = data[:,-1].astype(int)
data_train.shape, label_train.shape

((872, 5), (872,))

In [7]:
test_ = np.genfromtxt("data/bank-note/test.csv", dtype = float, delimiter = ',')
test_.shape

(500, 5)

In [8]:
test_ = np.hstack((np.ones((test_.shape[0],1)), test_))
test_.shape

(500, 6)

In [9]:
test = test_[:,:-1]
label = test_[:,-1].astype(int)
test.shape, label.shape

((500, 5), (500,))

In [10]:
def nll_(x, t, w):
    s_ = x @ w
    y_ = sigma(s_)
    I = (t == 0)
    y_[I] = 1-y_[I]
    y_ = np.log(y_+1e-10)
    return -y_.sum() + 0.5 * np.dot(w,w)

In [11]:
nll = lambda w: nll_(data_train, label_train, w)

In [12]:
d = data_train.shape[1]
mu = np.zeros(d)
M = np.eye(d)
M_inv = np.linalg.inv(M)

In [13]:
K = lambda r : 0.5 * r @ M_inv @ r

In [14]:
U = lambda w : nll(w)

In [15]:
def dU_(x, t, w):
    y = t - sigma(x@w)
    return w - x.T @ y

In [16]:
dU = lambda w : dU_(data_train, label_train, w)

In [17]:
H = lambda w, r : U(w) + K(r)

In [18]:
def Leapfrog(eps, w, r, dU = dU):
    r_half = r - eps * dU(w)/2
    w_new = w + eps * M_inv @ r_half
    r_new = r_half - eps * dU(w_new)/2
    return w_new, r_new

In [19]:
def L_step_Leapfrog(w, r, eps, L):
    r_ = r.copy()
    w_ = w.copy()
    for l in range(L):
        w_, r_ = Leapfrog(eps = eps, w = w_, r = r_)
    return w_ , r_

In [20]:
def HMC(w_0, r_pdf = Gaussian, L = 10, eps = 0.01, 
        n_sample = 10000, burn_in_after = 10**5, pick_every = 10, display = True):
    
    Sample_points = []
    iteration = -1
    c = -1
    sampled = 0
    w_new = w_0.copy()
    
    while sampled <= n_sample and iteration < 200000:
        iteration += 1
        r_0 = r_pdf(mu, M)
        w_new, r_new = L_step_Leapfrog(eps = eps, w = w_0, r = r_0, L = L)
        
        log_p = np.log(np.random.uniform(0,1))
        log_q = H(w_0, r_0) - H(w_new, -r_new) 
        if log_p <= log_q:
            c += 1
            if display and (iteration % 10000 == 0):
                print('Iteration is {}, number of accepted points is {}'.format(iteration, c))
                
            if iteration == burn_in_after:
                print('Burn-in stage started!')
                
            if iteration >= burn_in_after:
                if sampled % 10 == 0:
                    Sample_points.append(w_new.copy())
                    w_0 = w_new.copy()
                sampled += 1
            else:
                w_0 = w_new.copy()
                
    if len(Sample_points) == 0:
        Sample_points.append(w_new.copy())
        print(colored('acceptance rate is ZERO!!', 'red'))
                    
    return np.array(Sample_points), c/iteration

In [21]:
eps_list = [0.005, 0.01, 0.02, 0.05]
L_list = [10, 20, 50]

In [22]:
def predict(x, t, w, f = sigma):
    y = f(x @ w)
    l_pred = (y >= 0.5).astype(int)
    return (t == l_pred).mean()

In [23]:
def predictive_likelihood(x, t, w, f= sigma):
    y = f(x @ w)
    I = (t == 0)
    y[I] = 1 - y[I]
    return y.mean() 

In [24]:
def log_predictive_likelihood(x, t, w, f= sigma):
    y = f(x@w)
    I = (t == 0)
    y[I] = 1 - y[I]
    y = np.log(y)
    return y.mean() 

In [25]:
S_list = []
for eps in eps_list:
    for L in L_list:
        print('eps: {} and L: {}'.format(eps, L))
        t = time.time()
        S, p = HMC(w_0 = np.zeros(d), L = L, eps = eps, 
                   n_sample = 100, burn_in_after = 10**5, pick_every = 10, display = False)
        print('Running time for eps = {} and L = {} is {}'.format(eps, L, time.time()-t))
        print('Acceptance rate: {}'.format(p))
        S_list.append(S)
        
        av = 0
        av_log = 0
        mean = np.zeros(d)
        for i in range(len(S)):
            mean += S[i]/len(S)
            av_log += log_predictive_likelihood(x = test, t = label, w = S[i])
            av += predict(x = test, t = label, w = S[i])

        print(colored('Test predictive log-likelihood for eps = {} and L = {} is {}'.format(eps, L, 
                                                                                            av_log/len(S)), 'red'))
        print(colored('Test predictive accuracy for eps = {} and L = {} is {}'.format(eps, L, av/len(S)),'red'))
        print('Emperical mean is: ', mean)

eps: 0.005 and L: 10
Burn-in stage started!
Running time for eps = 0.005 and L = 10 is 517.7730922698975
Acceptance rate: 0.9984715284715284
Test predictive log-likelihood for eps = 0.005 and L = 10 is -0.031044802106972993
Test predictive accuracy for eps = 0.005 and L = 10 is 0.9887272727272726
Emperical mean is:  [ 2.89527225 -2.61487265 -1.48940474 -1.80841787 -0.08164588]
eps: 0.005 and L: 20
Burn-in stage started!
Running time for eps = 0.005 and L = 20 is 973.8321611881256
Acceptance rate: 0.9992607392607392
Test predictive log-likelihood for eps = 0.005 and L = 20 is -0.02993368366148391
Test predictive accuracy for eps = 0.005 and L = 20 is 0.9887272727272727
Emperical mean is:  [ 2.89267323 -2.73551013 -1.45113489 -1.78636197 -0.0788671 ]
eps: 0.005 and L: 50
Burn-in stage started!
Running time for eps = 0.005 and L = 50 is 2444.69638299942
Acceptance rate: 0.9988711288711288
Test predictive log-likelihood for eps = 0.005 and L = 50 is -0.03170123112682328
Test predictive acc

In [38]:
log_predictive_likelihood(x = test, t = label, w = np.zeros(d))

-0.6931471805599454

In [39]:
predict(x = test, t = label, w = np.zeros(d))

0.442

## (b) Gibbs sampling for the Bayesian probit model with augmented variables!

In [26]:
#truncnorm.rvs(a = 0, b= np.inf, loc=0, scale=1, size=1)

In [27]:
N, d = data_train.shape
S_inv = np.eye(d) + data_train.T @ data_train
S = np.linalg.inv(S_inv)

In [28]:
def _Z_update_(w, x, t):
    Z = np.zeros(N)
    mu = x @ w
    for i in range(N):
            Z[i] = (2*t[i]-1)*truncnorm.rvs(a = 0, b = np.inf, loc=mu[i], scale=1, size=1)
    return Z

In [29]:
def _Z_update(w, x, t):
    mu_ = x @ w
    mu_ *= (2*t-1)
    #a_ = np.zeros(N)
    b_ = np.inf * np.ones(N)
    Z = (2*t-1) * truncnorm.rvs(a = -mu_, b = b_, loc = mu_, scale=1)
    return Z

In [30]:
def _w_update(z):
    mu_ = S @ data_train.T @ z
    w = Gaussian(mean = mu_, cov = S)
    return w

In [31]:
def Gibbs_sampling(x = data_train, t = label_train, burn_in_at = 10**5, n_samples = 1000, pick_every  =10):
    w_ = np.random.normal(0,1, size = d)
    itr = 0
    while itr< burn_in_at:
        if itr % 10000 ==0:
            print('Iteration {} has started!'.format(itr+1))
        z_ = _Z_update(w_, x, t)
        w_ = _w_update(z_)
        itr += 1
    itr = 0
    w_Samples = []
    z_Samples = []
    print('burned in')
    while itr < n_samples * pick_every:
        if itr % pick_every == 0:
            w_Samples.append(w_.copy())
            z_Samples.append(z_.copy())
        z_ = _Z_update(w_, x, t)
        w_ = _w_update(z_)
        itr += 1
    return w_Samples, z_Samples

In [32]:
t = time.time()
W, Z = Gibbs_sampling(burn_in_at = 10**5, n_samples = 1000, pick_every  =10)
print(time.time()-t)

Iteration 1 has started!
Iteration 10001 has started!
Iteration 20001 has started!
Iteration 30001 has started!
Iteration 40001 has started!
Iteration 50001 has started!
Iteration 60001 has started!
Iteration 70001 has started!
Iteration 80001 has started!
Iteration 90001 has started!
burned in
4483.266856908798


In [36]:
acc = 0
pll = 0
K = len(W)
for w in W:
    pll += log_predictive_likelihood(x = test, t = label, w = W[i], f=multivariate_normal.cdf)/K
    acc += predict(x = test, t = label, w = W[i], f=multivariate_normal.cdf)/K

In [37]:
acc, pll

(0.9879999999999906, -0.02897856072681373)

In [35]:
np.array(W).mean(axis = 0)

array([ 2.18676179, -2.14897079, -1.30251199, -1.5515661 , -0.1651304 ])